In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('./roast.zip')
zip_ref.extractall('./')


In [ ]:
import json
import re
import os
import tensorflow as tf
tf.enable_eager_execution()
import numpy as np

In [ ]:
corpus = ''
for file in os.listdir('./roast'):
    fp = open('./roast/' + file, 'r')
    z = json.load(fp)
    sentences = list(z.values())[0]
    for s in sentences:
        corpus += s

In [ ]:
len(corpus)

In [ ]:
corpus_filtered = re.sub('[^a-z0-9A-Z\s]', ' ', corpus)

In [ ]:
corpus_filtered = re.sub('\'', '', corpus_filtered)

In [ ]:
corpus_filtered = corpus_filtered.lower()

In [ ]:
corpus_filtered = re.sub('\s+', ' ', corpus_filtered)

In [ ]:
data = corpus_filtered

In [ ]:
id2c = list(set(data))
c2id = {c:i for i,c in enumerate(id2c)}

In [ ]:
def encode_string(string_in):
    return np.array([c2id[c] for c in string_in])

def decode_string(encoded_str):
    return ''.join([id2c[c] for c in encoded_str])

In [ ]:
batch_size = 32
seq_length = 20
vocab_size = len(id2c)
data_size = len(data)

i_size = vocab_size
hidden_state_size = 30

In [ ]:
def get_batch(X, index_list, seq_length=seq_length, data_size=data_size):
    batch = []
    y = []
    for i in index_list:
        start = i%data_size
        end = (i + seq_length)%data_size
        if(end - start == seq_length):
            batch.append(encode_string(X[start:end]))
            y.append(encode_string(X[start+1:end+1]))
        else:
            batch.append(encode_string(X[start:]+X[:end]))
            y.append(encode_string(X[start+1:]+X[:end+1]))            
            
    return np.array(batch), np.array(y)
    

In [ ]:
w_ii = tf.Variable(tf.random.truncated_normal([i_size, hidden_state_size]))
w_io = tf.Variable(tf.random.truncated_normal([hidden_state_size, hidden_state_size]))
b_i = tf.Variable(tf.random.truncated_normal([hidden_state_size]))

w_oi = tf.Variable(tf.random.truncated_normal([i_size, hidden_state_size]))
w_oo = tf.Variable(tf.random.truncated_normal([hidden_state_size, hidden_state_size]))
b_o = tf.Variable(tf.random.truncated_normal([hidden_state_size]))

w_mi = tf.Variable(tf.random.truncated_normal([i_size, hidden_state_size]))
w_mo = tf.Variable(tf.random.truncated_normal([hidden_state_size, hidden_state_size]))
b_m = tf.Variable(tf.random.truncated_normal([hidden_state_size]))

w_fi = tf.Variable(tf.random.truncated_normal([i_size, hidden_state_size]))
w_fo = tf.Variable(tf.random.truncated_normal([hidden_state_size, hidden_state_size]))
b_f = tf.Variable(tf.random.truncated_normal([hidden_state_size]))    
    
w_out = tf.Variable(tf.random.truncated_normal([hidden_state_size, vocab_size]))
b_out = tf.Variable(tf.random.truncated_normal([vocab_size])) 

def lstm_cell(i, o, state):
    input_gate = tf.sigmoid(i @ w_ii + o @ w_io  + b_i);
    output_gate = tf.sigmoid(i @ w_oi + o @ w_oo + b_o);
    forget_gate = tf.sigmoid(i @ w_fi + o @ w_fo + b_f);
    memory_gate = tf.tanh(i @ w_mi + o @ w_mo + b_m);

    state = forget_gate * state + input_gate * memory_gate 
    output = output_gate * tf.tanh(state)

    return output, state

In [ ]:
#train network
#X = tf.placeholder(tf.int64, [batch_size, seq_length])
#y = tf.placeholder(tf.int64, [batch_size, seq_length])

def get_probabilities_logits(X):
    encoded_X = tf.one_hot(X, vocab_size, axis=-1)
    output_sequence = []

    h_prev = tf.zeros([X.shape[0], hidden_state_size])
    o_tprev = tf.zeros([X.shape[0], hidden_state_size])
    
    for t in range(X.shape[1]):
        x_t = encoded_X[:, t, :]  # column t
        o_tnext, h_next = lstm_cell(x_t, o_tprev, h_prev)
        o_tprev = o_tnext#tf.argmax(probabs_next)
        h_prev = h_next
        output_sequence.append(o_tnext)
        
    chars_output = tf.reshape(output_sequence, shape=[-1, hidden_state_size]) @ w_out + b_out
    chars_output = tf.reshape(chars_output, shape=[X.shape[1], X.shape[0], -1])
    chars_output = tf.transpose(chars_output, perm=[1, 0, 2])
#    print(chars_output) 
    return chars_output